In [5]:
from google.auth import default
credentials, project = default()

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
from pathlib import Path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from dotenv import load_dotenv
load_dotenv()

from src.config_loader import config
from src.translation import review_translations_with_anthropic, review_translated_phrases_batch
from src.utils import get_first_n_items
from src.gcs_storage import get_translated_phrases_path, read_from_gcs, upload_to_gcs
config.TARGET_LANGUAGE_NAME

# Translation Refinement

## Issues with Google Translate

It can sound stiff / formal and be more related to written text rather than verbal.

## Approach

Use Sonnet 3.5 via the Anthropic API, using a tool to return adjusted translations in JSON format for re-uploading to our phrase translation store (json file)

In [7]:
# get current translated JSON file
translated_file_path =get_translated_phrases_path(collection="LM1000")

phrase_translations = read_from_gcs(bucket_name=config.GCS_PRIVATE_BUCKET, file_path=translated_file_path)

In [24]:
test_dict = get_first_n_items(phrase_translations, 20)

In [ ]:
len(phrase_translations)

In [ ]:
improved_translations = review_translated_phrases_batch(phrase_translations, model = "claude-3-5-sonnet-latest")

In [ ]:
len(improved_translations)

In [ ]:
upload_to_gcs(obj=improved_translations, bucket_name=config.GCS_PRIVATE_BUCKET, file_name=translated_file_path)

In [ ]:
improved_translations